In [1]:
import boto3
ec2 = boto3.client('ec2')
elb = boto3.client('elbv2')

### [1] Create 2 EC2 instances in two different availability zones of a specific region. 

In [2]:
KEY_NAME = "22962256-key"
SG_NAME = "22962256-sg"
ELB_NAME = "22962256-elb"
TARGET_GROUP_NAME = "22962256-tg"
# Prepare subnet for ec2 creation
VPC_ID = "vpc-00da1b229d10a51b6"
SUBNETS = [
    {"id": "subnet-0a7d8e51199753df1", "name": "ap-southeast-2a"},
    {"id": "subnet-0c1878c6a739707b7", "name": "ap-southeast-2b"},
]

In [3]:
# Create Security Group
sg_response = ec2.create_security_group(
    Description="security group for development environment by boto3",
    GroupName=SG_NAME,
    VpcId=VPC_ID,
)

sg_id = sg_response["GroupId"]

ec2.authorize_security_group_ingress(
    GroupId=sg_id,
    IpPermissions=[
        {
            "IpProtocol": "tcp",
            "FromPort": 22,
            "ToPort": 22,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}],
        },
        {
            "IpProtocol": "tcp",
            "FromPort": 80,
            "ToPort": 80,
            "IpRanges": [{"CidrIp": "0.0.0.0/0"}],
        },
    ],
)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-058ba5cd452a45ae8',
   'GroupId': 'sg-0678e728c9bab2d84',
   'GroupOwnerId': '489389878001',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-0873192b53f9d2bfc',
   'GroupId': 'sg-0678e728c9bab2d84',
   'GroupOwnerId': '489389878001',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': '67d763c3-2c90-4159-8dc5-72568803e86e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '67d763c3-2c90-4159-8dc5-72568803e86e',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1124',
   'date': 'Mon, 25 Sep 2023 02:49:19 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In the following step, we launch EC2 instances and install Apache web server on them. We will use these instances as targets for our load balancer.
[Run commands on your Linux instance at launch](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/user-data.html)

In [4]:
def get_userdata(name):
    return (
        """
            #cloud-config
            runcmd:
            - apt-get update
            - apt install apache2 -y
            - chmod 777 /var/www/html/index.html
            - echo "This is page from 22962256-%s." > /var/www/html/index.html
        """
        % name
    )


instance_responces = list(
    map(
        lambda subnet: ec2.run_instances(
            ImageId="ami-d38a4ab1",
            InstanceType="t2.micro",
            MaxCount=1,
            MinCount=1,
            KeyName=KEY_NAME,
            SubnetId=subnet["id"],
            SecurityGroupIds=[sg_id],
            UserData=get_userdata(subnet["name"]),
            TagSpecifications=[
                {
                    "ResourceType": "instance",
                    "Tags": [
                        {"Key": "Name", "Value": "22962256-" + subnet["name"]},
                    ],
                },
            ],
        ),
        SUBNETS,
    )
)

In [5]:
instance_ids = list(
    map(lambda response: response["Instances"][0]["InstanceId"], instance_responces)
)

### [2] Create the Application Load Balancer.
**[a] Create the load balancer and specify the two region subnets and a security group** 

In [6]:
load_balancer_response = elb.create_load_balancer(
    Name=ELB_NAME,
    Subnets=list(map(lambda subnet: subnet["id"], SUBNETS)),
    SecurityGroups=[sg_id],
    Scheme="internet-facing",
    IpAddressType="ipv4",
)


**[c] Create a target group using the same VPC that you used to create the instances** 

In [8]:
target_group_response = elb.create_target_group(
    Name=TARGET_GROUP_NAME,
    Protocol="HTTP",
    Port=80,
    VpcId=VPC_ID,
    HealthCheckProtocol="HTTP",
    HealthCheckEnabled=True,
    HealthCheckIntervalSeconds=30,
    HealthCheckTimeoutSeconds=5,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=2,
    TargetType="instance",
)
target_group_arn = target_group_response["TargetGroups"][0]["TargetGroupArn"]

**[b] Create a listener with a default rule Protocol: HTTP and Port 80 forwarding on to the target group** 

In [9]:
response = elb.create_listener(
    LoadBalancerArn=load_balancer_response["LoadBalancers"][0]["LoadBalancerArn"],
    Protocol='HTTP',
    Port=80,
    DefaultActions=[
        {
            'Type': 'forward',
            'ForwardConfig': {
                'TargetGroups': [
                    {
                        'TargetGroupArn': target_group_arn,
                    },
                ],
            }
        },
    ],
)

**[d] Register targets in the target group**

In [10]:
register_targets_response = elb.register_targets(
    TargetGroupArn=target_group_arn,
    Targets=list(map(lambda id: {"Id": id, "Port": 80}, instance_ids)),
)

### Clean-up

In [12]:
# Clean up
ec2.terminate_instances(InstanceIds=instance_ids)
elb.delete_load_balancer(LoadBalancerArn=load_balancer_response["LoadBalancers"][0]["LoadBalancerArn"])
ec2.delete_security_group(GroupId=sg_id)

ClientError: An error occurred (DependencyViolation) when calling the DeleteSecurityGroup operation: resource sg-0678e728c9bab2d84 has a dependent object